In [10]:
import torch
import os
from torchvision import datasets, models, transforms
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
import neptune
import matplotlib.gridspec as gridspec
cudnn.benchmark = True
from sklearn.metrics import confusion_matrix

In [11]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [12]:
device = torch.device("cpu")

In [13]:
model_ft = models.resnet34()

num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 10)
model_ft = model_ft.to(device)
model_ft.load_state_dict(torch.load('weights-resnet34-bestloss.pth',map_location ='cpu'))

<All keys matched successfully>

In [14]:
nb_classes = 10

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device=device)
lbllist=torch.zeros(0,dtype=torch.long, device=device)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        from sklearn.metrics import classification_report

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])
target_names = ['ясно' ,'облачно' ,'туман' ,'мороз' ,'град' ,'молния' , 'дождь', 'радуга',
           'снег', 'восход']
print(classification_report(lbllist.numpy(), predlist.numpy(),target_names=target_names))

              precision    recall  f1-score   support

        ясно       0.67      0.64      0.65        99
     облачно       0.66      0.67      0.67       129
       туман       0.80      0.83      0.82       147
       мороз       0.76      0.79      0.77       165
        град       0.92      0.72      0.81        64
      молния       1.00      0.79      0.88        43
       дождь       0.72      0.74      0.73       177
      радуга       1.00      0.75      0.86        28
        снег       0.62      0.71      0.66       162
      восход       0.92      0.86      0.89        69

    accuracy                           0.75      1083
   macro avg       0.81      0.75      0.77      1083
weighted avg       0.75      0.75      0.75      1083

